In [1]:
import numpy as np
import pandas as pd

In [25]:
year = 2017
election_round = "runoff" # first_round, runoff

In [26]:
df = pd.read_excel(f"data/Chile/{year}_{election_round}.xlsx")
df.head()

,Región,Provincia,Circ.Senatorial,Distrito,Comuna,Circ.Electoral,Local,Mesa,Tipo mesa,Mesas Fusionadas,Electores,Nro.voto,Candidato,Votos TRICEL
0,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,1.0,M,1M-5M,419.0,3.0,SEBASTIAN PIÑERA ECHENIQUE,130.0
1,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,1.0,M,1M-5M,419.0,4.0,ALEJANDRO GUILLIER ALVAREZ,28.0
2,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,1.0,M,1M-5M,419.0,901.0,Votos Nulos,0.0
3,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,1.0,M,1M-5M,419.0,900.0,Votos Blancos,1.0
4,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,2.0,M,2M-3M,441.0,3.0,SEBASTIAN PIÑERA ECHENIQUE,114.0


In [27]:
# list(df)

In [28]:
if year == 2017:
    df["polling_id"] = df["Comuna"] + "-" + df["Circ.Electoral"] + "-" + df["Local"] + "-" + df["Mesa"].fillna(0).astype(int).astype(str)\
        + "-" + df["Tipo mesa"].fillna("NA") + "-" + df["Mesas Fusionadas"].astype(str)

elif year == 2021:
    df["polling_id"] = df["Comuna"] + "-" + df["Circ. Electoral"] + "-" + df["Local"] + "-" + df["Nro. Mesa"].fillna(0).astype(int).astype(str)\
        + "-" + df["Tipo de mesa"].fillna("NA").astype(str) + "-" + df["Mesas Fusionadas"].astype(str)

In [29]:
df.Candidato.unique()

array(['SEBASTIAN PIÑERA ECHENIQUE', 'ALEJANDRO  GUILLIER ALVAREZ',
       'Votos Nulos', 'Votos Blancos', nan, 'TOTALES'], dtype=object)

In [30]:
df = df[~df["Candidato"].isin(["VOTOS NULOS", "VOTOS EN BLANCO", "Votos Blancos", "Votos Nulos", "TOTALES", np.nan])]

In [31]:
df = df.dropna(axis=1, how="all")

df["Votos TRICEL"] = df["Votos TRICEL"].fillna(0)
df["rank"] = df.groupby(["polling_id"])["Votos TRICEL"].rank("min", ascending=False).astype(int)

In [32]:
tt = df.groupby(["polling_id", "Candidato"]).agg({"Votos TRICEL": "sum"})
tt["rate"] = tt.groupby(level=[0]).apply(lambda x: x/x.sum())
tt = tt.reset_index()
tt = tt[["polling_id", "Candidato", "rate"]]

In [33]:
df = pd.merge(df, tt, on=["polling_id", "Candidato"])

In [34]:
df_filtered = df[["polling_id", "Candidato", "Votos TRICEL", "rank", "rate"]]\
    .rename(columns={"Candidato": "candidate", "Votos TRICEL": "value"})

In [35]:
df_filtered["candidate"] = df_filtered["candidate"].apply(lambda x: x.strip())

In [36]:
df_filtered.candidate.unique()

array(['SEBASTIAN PIÑERA ECHENIQUE', 'ALEJANDRO  GUILLIER ALVAREZ'],
      dtype=object)

In [37]:
print(df_filtered.shape)
df_filtered.head()

(85780, 5)


,polling_id,candidate,value,rank,rate
0,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-1-M-1M-5M,SEBASTIAN PIÑERA ECHENIQUE,130.0,1,0.822785
1,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-1-M-1M-5M,ALEJANDRO GUILLIER ALVAREZ,28.0,2,0.177215
2,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-2-M-2M-3M,SEBASTIAN PIÑERA ECHENIQUE,114.0,1,0.802817
3,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-2-M-2M-3M,ALEJANDRO GUILLIER ALVAREZ,28.0,2,0.197183
4,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-2-V-2V-3V-4V,SEBASTIAN PIÑERA ECHENIQUE,90.0,1,0.737705


In [38]:
df_filtered.to_csv(f"data_output/Chile/{year}_{election_round}.csv.gzip", compression="gzip", index=False)

In [39]:
df

,Región,Provincia,Circ.Senatorial,Distrito,Comuna,Circ.Electoral,Local,Mesa,Tipo mesa,Mesas Fusionadas,Electores,Nro.voto,Candidato,Votos TRICEL,polling_id,rank,rate
0,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,1.0,M,1M-5M,419.0,3.0,SEBASTIAN PIÑERA ECHENIQUE,130.0,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-1-M-1M-5M,1,0.822785
1,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,1.0,M,1M-5M,419.0,4.0,ALEJANDRO GUILLIER ALVAREZ,28.0,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-1-M-1M-5M,2,0.177215
2,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,2.0,M,2M-3M,441.0,3.0,SEBASTIAN PIÑERA ECHENIQUE,114.0,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-2-M-2M-3M,1,0.802817
3,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,2.0,M,2M-3M,441.0,4.0,ALEJANDRO GUILLIER ALVAREZ,28.0,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-2-M-2M-3M,2,0.197183
4,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,2.0,V,2V-3V-4V,383.0,3.0,SEBASTIAN PIÑERA ECHENIQUE,90.0,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-2-V-2V-3V-4V,1,0.737705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85775,15.0,PARINACOTA,1a Circunscripción,1er Distrito,PUTRE,PUTRE,LICEO GRANADEROS,20.0,V,20V-21V-22V,403.0,4.0,ALEJANDRO GUILLIER ALVAREZ,26.0,PUTRE-PUTRE-LICEO GRANADEROS-20-V-20V-21V-22V,2,0.333333
85776,15.0,PARINACOTA,1a Circunscripción,1er Distrito,PUTRE,PUTRE,LICEO GRANADEROS,23.0,V,23V-24V-25V,394.0,3.0,SEBASTIAN PIÑERA ECHENIQUE,46.0,PUTRE-PUTRE-LICEO GRANADEROS-23-V-23V-24V-25V,1,0.613333
85777,15.0,PARINACOTA,1a Circunscripción,1er Distrito,PUTRE,PUTRE,LICEO GRANADEROS,23.0,V,23V-24V-25V,394.0,4.0,ALEJANDRO GUILLIER ALVAREZ,29.0,PUTRE-PUTRE-LICEO GRANADEROS-23-V-23V-24V-25V,2,0.386667
85778,15.0,PARINACOTA,1a Circunscripción,1er Distrito,PUTRE,PUTRE,LICEO GRANADEROS,26.0,V,26V-27V,261.0,3.0,SEBASTIAN PIÑERA ECHENIQUE,33.0,PUTRE-PUTRE-LICEO GRANADEROS-26-V-26V-27V,1,0.568966


In [40]:
list(df)

['Región',
 'Provincia',
 'Circ.Senatorial',
 'Distrito',
 'Comuna',
 'Circ.Electoral',
 'Local',
 'Mesa',
 'Tipo mesa',
 'Mesas Fusionadas',
 'Electores',
 'Nro.voto',
 'Candidato',
 'Votos TRICEL',
 'polling_id',
 'rank',
 'rate']

In [41]:
year

2017

In [42]:
if year == 2021:

    df_location = df[["polling_id", "Nro. Región", "Región", "Provincia", "Circ. Senatorial", "Distrito", "Comuna", "Circ. Electoral", "Local",
        "Nro. Mesa", "Tipo de mesa", "Mesas Fusionadas", "Electores"]].drop_duplicates()

    df_location = df_location.rename(columns={
        "Nro. Región": "region_id",
        "Región": "region",
        "Provincia": "province",
        "Circ. Senatorial": "senate_district",
        "Distrito": "district",
        "Comuna": "commune",
        "Circ. Electoral": "electoral_circ",
        "Local": "polling_place",
        "Nro. Mesa": "polling_table",
        "Tipo de mesa": "polling_type",
        "Mesas Fusionadas": "mixed_polling",
        "Electores": "voters"
    })
    
    for col in ["region_id", "senate_district", "district", "polling_table", "voters"]:
        df_location[col] = df_location[col].astype(int)
    
elif year == 2017:
    df_location = df[["polling_id", "Región", "Provincia", "Circ.Senatorial", "Distrito", "Comuna", "Circ.Electoral", "Local",
        "Mesa", "Tipo mesa", "Mesas Fusionadas", "Electores"]].drop_duplicates()

    df_location = df_location.rename(columns={
        "Región": "region_id",
        "Provincia": "province",
        "Circ.Senatorial": "senate_district",
        "Distrito": "district",
        "Comuna": "commune",
        "Circ.Electoral": "electoral_circ",
        "Local": "polling_place",
        "Mesa": "polling_table",
        "Tipo mesa": "polling_type",
        "Mesas Fusionadas": "mixed_polling",
        "Electores": "voters"
    })
    
    for col in ["region_id", "polling_table", "voters"]:
        df_location[col] = df_location[col].astype(int)

df_location.to_csv(f"data_output/Chile/{year}_{election_round}_location.csv.gzip", compression="gzip", index=False)